In [1]:
import pandas as pd
import numpy as np
from numpy import isnan
from collections import Counter
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

from computeNutriScore import computeNutriScore

DATA_FOLDER = './data/'

In [6]:
# Load Data and Clean
data_raw = pd.read_csv(DATA_FOLDER + 'en.openfoodfacts.org.products.csv', delimiter = '\t', low_memory = False)
data_clean = data_raw.copy()

# Drop duplicates according to the code
data_clean.drop_duplicates(subset = 'code', keep = False, inplace = True)
data_clean.set_index('code', inplace = True)

# Drop product without product name and without tags
data_clean = data_clean[~data_clean.product_name.isna() & ~data_clean.categories_tags.isna()]

In [7]:
# Create 2 DataFrame - one for only food (useful for compute NutriScore) and an other one with the left columns
column_for_data_food = ['product_name','categories_tags','energy_100g','fat_100g','saturated-fat_100g',
                                            'sugars_100g','salt_100g','sodium_100g','fruits-vegetables-nuts_100g',
                                            'fruits-vegetables-nuts-estimate_100g','fiber_100g','proteins_100g']

data_food = data_clean[column_for_data_food]

data_clean.drop(column_for_data_food, axis = 1, inplace = True)


In [24]:
data_food.describe(percentiles = [.25, .50, .75, .99])

,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,salt_100g,sodium_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,fiber_100g,proteins_100g
count,1.480730e+05,146489.000000,143742.000000,144151.000000,142650.000000,142680.000000,2939.000000,7466.000000,78241.000000,147262.000000
mean,1.200323e+03,13.714214,5.403005,13.973513,1.115041,0.453263,43.438452,43.996718,2.703701,7.017626
std,1.284265e+04,18.868590,8.866378,19.734165,4.775650,2.117850,33.508329,29.703503,4.547284,7.920598
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.470000e+02,0.500000,0.100000,1.000000,0.030000,0.011811,12.000000,14.000000,0.000000,0.900000
50%,9.950000e+02,5.700000,1.680000,4.500000,0.350000,0.139382,43.000000,50.000000,1.500000,5.100000
75%,1.648000e+03,21.100000,7.400000,19.000000,1.150620,0.456693,65.000000,63.000000,3.400000,9.700000
99%,3.700000e+03,93.000000,42.000000,82.000000,10.000000,3.937008,100.000000,100.000000,20.000000,30.339000
max,2.790000e+06,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [18]:
def Complete_ratio_column(df):
    for column in df.columns:
        ratio = len(df[[column]].dropna()) * 100/len(df)
        print("For {}, {} % is complete".format(column, round(ratio, 3)))
        
Complete_ratio_column(data_food)

For product_name, 100.0 % is complete
For categories_tags, 100.0 % is complete
For energy_100g, 86.001 % is complete
For fat_100g, 85.081 % is complete
For saturated-fat_100g, 83.486 % is complete
For sugars_100g, 83.724 % is complete
For salt_100g, 82.852 % is complete
For sodium_100g, 82.869 % is complete
For fruits-vegetables-nuts_100g, 1.707 % is complete
For fruits-vegetables-nuts-estimate_100g, 4.336 % is complete
For fiber_100g, 45.443 % is complete
For proteins_100g, 85.53 % is complete


In [25]:
# Put NaN for incoherent values
def Coherent_value(df):
    """ Compute coherent value for food values """
    for i in range(len(df)):
        # energy
        value = df.iat[i, 2]
        if  value < 0 or value > 4000:
            df.iat[i, 2] = np.nan
        
        #other column
        for column in range(3, len(df.columns)):
            value = df.iat[i, column]
            if (value < 0) or (value > 100):
                df.iat[i, column] = np.nan
    return df

data_food = Coherent_value(data_food)

In [26]:
data_food.describe()

,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,salt_100g,sodium_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,fiber_100g,proteins_100g
count,147879.000000,146489.000000,143742.000000,144151.000000,142650.000000,142680.000000,2939.000000,7466.000000,78241.000000,147262.000000
mean,1090.190054,13.714214,5.403005,13.973513,1.115041,0.453263,43.438452,43.996718,2.703701,7.017626
std,826.969127,18.868590,8.866378,19.734165,4.775650,2.117850,33.508329,29.703503,4.547284,7.920598
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,347.000000,0.500000,0.100000,1.000000,0.030000,0.011811,12.000000,14.000000,0.000000,0.900000
50%,992.000000,5.700000,1.680000,4.500000,0.350000,0.139382,43.000000,50.000000,1.500000,5.100000
75%,1644.000000,21.100000,7.400000,19.000000,1.150620,0.456693,65.000000,63.000000,3.400000,9.700000
max,3987.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [31]:
all_tags = [tags.split(',') for tags in list(data_food[['categories_tags']].dropna().categories_tags)]

list_tags = []
for tags in all_tags:
    for tag in tags:
        list_tags.append(tag)
        
dic_tag = dict(Counter(list_tags).most_common())

In [32]:
def relevant_tag(dic, tags, threshold = 5):
    """ Return the least frequent tag from the list, who however has more than 'threshold' 
    correspondancies in the database """
    dict_ = {}
    sort = []
    
    for tag in tags:
        dict_[tag] = dic[tag]
        
    dict_ = {k: v for k, v in dict_.items() if v >= threshold}
    if len(dict_) > 0:
        sort = sorted(dict_, key=dict_.get, reverse=False)
        return sort
    else :
        return None

In [37]:
data_food_left = data_food[(data_food.isnull().sum(axis=1) == 10) & ~(data_food.categories_tags.str.contains('beverages', case = False))]

list_tag = []
for index in range(len(data_food_left)):
    sort = relevant_tag(dic_tag, data_food_left.iloc[index].categories_tags.split(','), 1)
    list_tag.append(sort[0])

dict(Counter(list_tag).most_common())
    

{'en:dietary-supplements': 141,
 'en:honeys': 113,
 'en:cheeses': 105,
 'en:yogurts': 87,
 'en:candies': 83,
 'en:biscuits': 83,
 'en:chocolates': 79,
 'en:free-range-eggs': 69,
 'en:sauces': 66,
 'fr:miels-d-acacia': 66,
 'en:dark-chocolates': 65,
 'en:mustards': 57,
 'en:eggs': 56,
 'en:chicken-breasts': 56,
 'en:cakes': 49,
 'en:honeys-from-france': 48,
 'fr:miels-cremeux': 48,
 'en:groceries': 46,
 'en:prepared-meats': 42,
 'en:crackers': 41,
 'en:beef': 41,
 'en:pastries': 41,
 'fr:miels-de-fleurs': 41,
 'en:condiments': 39,
 'fr:foies-gras-entiers': 38,
 'fr:tuiles-salees': 36,
 'en:fresh-meats': 36,
 'en:sausages': 34,
 'en:milk-chocolates': 33,
 'en:food-colorings': 33,
 'en:vinegars': 33,
 'en:pork': 32,
 'en:dried-meals': 32,
 'en:milks': 32,
 'en:bieres': 32,
 'en:salts': 32,
 'en:cage-eggs': 32,
 'en:white-hams': 30,
 'en:balsamic-vinegars-of-modena': 30,
 'en:cocoa-powder': 29,
 'en:bonbons': 29,
 'en:ketchup': 27,
 'en:chocolate-biscuits': 27,
 'en:confectioneries': 27,
 

In [38]:
data_food = data_food[~(data_food.isnull().sum(axis=1) == 10) & 
                      (data_food.categories_tags.str.contains('beverages', case = False))]
Complete_ratio_column(data_food)

For product_name, 100.0 % is complete
For categories_tags, 100.0 % is complete
For energy_100g, 99.111 % is complete
For fat_100g, 97.675 % is complete
For saturated-fat_100g, 95.496 % is complete
For sugars_100g, 96.303 % is complete
For salt_100g, 94.43 % is complete
For sodium_100g, 94.448 % is complete
For fruits-vegetables-nuts_100g, 3.353 % is complete
For fruits-vegetables-nuts-estimate_100g, 6.948 % is complete
For fiber_100g, 60.313 % is complete
For proteins_100g, 98.397 % is complete


In [40]:
data_food_left = data_food[data_food.fiber_100g.isna()]

list_tag = []
for index in range(len(data_food_left)):
    sort = relevant_tag(dic_tag, data_food_left.iloc[index].categories_tags.split(','), 1)
    list_tag.append(sort[0])

dict(Counter(list_tag).most_common())
    

{'en:non-alcoholic-beverages': 2122,
 'en:sugared-beverages': 858,
 'en:artificially-sweetened-beverages': 344,
 'en:non-sugared-beverages': 305,
 'en:extra-virgin-olive-oils': 263,
 'en:squeezed-apple-juices': 246,
 'en:strawberry-jams': 237,
 'en:apricot-jams': 196,
 'en:sodas': 182,
 'en:fruit-based-beverages': 178,
 'en:colas': 178,
 'en:lemonades': 156,
 'en:bilberries-jams': 156,
 'en:iced-teas': 137,
 'en:potato-crisps-in-sunflower-oil': 135,
 'en:raspberry-jams': 134,
 'en:multifruit-juices': 133,
 'en:flavoured-potato-crisps': 124,
 'en:cherry-jams': 118,
 'en:energy-drinks': 116,
 'en:squeezed-orange-juices': 113,
 'en:figs-jams': 112,
 'en:margarines': 95,
 'en:rices': 94,
 'en:drinkable-yogurts': 93,
 'en:natural-mineral-waters': 85,
 'en:fruit-juices': 81,
 'en:potato-crisps': 81,
 'en:breads': 80,
 'en:apple-juices': 79,
 'en:mixed-fruit-jams': 77,
 'en:fruit-sodas': 76,
 'en:mineral-waters': 76,
 'en:chestnut-spreads': 73,
 'en:hummus': 72,
 'en:blackberry-jams': 72,
 'e